In [1]:
# SVM  best 2500 cu CountVectorizer

In [2]:
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import f1_score
import numpy as np

In [3]:
def normalize_data(train_data, test_data, type=None):
    scaler = None
    if type == 'standard':
        scaler = preprocessing.StandardScaler()

    elif type == 'l1':
        scaler = preprocessing.Normalizer(norm='l1')

    elif type == 'l2':
        scaler = preprocessing.Normalizer(norm='l2')

    if scaler is not None:
        scaler.fit(train_data)
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data) 
        return (scaled_train_data, scaled_test_data)
    else:
        print("No scaling was performed. Raw data is returned.")
        return (train_data, test_data)

In [4]:
def accuracy_score(y_true, y_pred):
    return (y_true == y_pred).mean()
#train_labels = train_labels.astype(int)

In [5]:
def get_sentence(data):
    new_data = []
    
    for item in data:
        new_data.append(item[1])
        
    return np.array(new_data)

In [6]:
def get_labels(data):
    new_data = []
    
    for item in data:
        new_data.append(item[1])
    return np.array(new_data)

In [7]:
# load data
training_data = np.genfromtxt('data_aliens/train_samples.txt', dtype='str', comments='\\\\\\\\\\\\\\\\', delimiter='\t', invalid_raise = False)
training_labels = np.loadtxt('data_aliens/train_labels.txt', dtype='int')

validation_data = np.genfromtxt('data_aliens/validation_samples.txt', dtype='str', comments='\\\\\\\\\\\\\\\\', delimiter='\t', invalid_raise = False)
validation_labels = np.loadtxt('data_aliens/validation_labels.txt', dtype='int')

merged_data = np.concatenate((training_data, validation_data), axis=0)
merged_labels = np.concatenate((training_labels, validation_labels), axis=0)

x_train = get_sentence(training_data)
y_train = get_labels(training_labels)

x_valid = get_sentence(validation_data)
y_valid = get_labels(validation_labels)

x_merged = get_sentence(merged_data)
y_merged = get_labels(merged_labels)

print(validation_data.shape)

# print(training_data)
# print(get_labels(training_labels))

(5000, 2)


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(lowercase = False, max_features=3000, strip_accents=None, min_df=1, stop_words=None)
X_train_counts = count_vect.fit_transform(x_train)
names = count_vect.get_feature_names_out()
X_test_counts = count_vect.transform(x_valid)
#test_names = count_vect.get_feature_names_out()

print(X_train_counts.shape)

print(X_train_counts.toarray())

train_data = X_train_counts.toarray()
test_data = X_test_counts.toarray()

#print(X_test_counts.shape)
#count_vect.get_feature_names_out()


(10000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
#data frame
import pandas as pd

def final_features(row, vect, names):
    df = pd.DataFrame(data=vect,
                  columns = names)
    
    txt = pd.DataFrame(row, columns = ["data"])
    
    vowels=['a','e','i','o','u']
    
    punc = ['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?"]
    
    def vowels_density(txt):
        words = len(txt.split(" "))
        chars = len(txt.replace(" ", ""))
        
        nr_vowels = sum([1 for c in txt if c in vowels])
        
        return(nr_vowels / chars)
    
    def punc_density(txt):
        words = len(txt.split(" "))
        chars = len(txt.replace(" ", ""))
        
        nr_punc = sum([1 for c in txt if c in punc])
        
        return(nr_punc / chars)
    #print("denitate vocale: ", txt["data"].apply(lambda txt: vowels_density(txt)))
    
    #df["num_question_marks"] = txt["data"].apply(lambda txt: txt.count("?"))
    df['punc_density'] = txt["data"].apply(lambda txt: punc_density(txt))
    df['vowels_density'] = txt["data"].apply(lambda txt: vowels_density(txt))
    df['num_unique_words'] = txt['data'].apply(lambda txt: len(set(w for w in txt.split())))
    #df['capitals'] = txt['data'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    #print(df["num_question_marks"])
    print("DF",df)
    return df

In [10]:
# import pandas as pd

# df = pd.DataFrame(data=train_data,
#                   columns = count_vect.get_feature_names_out())
# print("DF",df)



# txt = pd.DataFrame(x_train, columns = ["data"])

# # def counter(txt):
# #     nr = 0
# #     for i in range(0,len(txt)):
# #         nr txt[i]["data"] == '*':
# #             nr += 1
# #     return nr
# df["num_question_marks"] = txt["data"].apply(lambda txt: txt.count("?"))
# print(df["num_question_marks"])

In [11]:
# #valid

# valid_df = pd.DataFrame(data=test_data,
#                   columns = count_vect.get_feature_names_out())

# txt = pd.DataFrame(x_valid, columns = ["data"])

# valid_df["num_question_marks"] = txt["data"].apply(lambda txt: txt.count("?"))
# print(df["num_question_marks"])

train_df = final_features(x_train, train_data, names)

valid_df = final_features(x_valid, test_data, names)


DF       00  000  0002  000g001  001  002  003  004  005  006  ...  țkK  țlrȚ  \
0      0    0     0        0    0    0    0    0    0    0  ...    0     0   
1      0    0     0        0    0    0    0    0    0    0  ...    0     0   
2      1    0     0        0    0    0    0    0    0    0  ...    0     0   
3      0    0     0        0    0    0    0    0    0    0  ...    0     0   
4      0    0     0        0    0    0    0    0    0    0  ...    0     0   
...   ..  ...   ...      ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
9995   0    0     0        0    0    0    0    0    0    0  ...    0     0   
9996   0    0     0        0    0    0    0    0    0    0  ...    0     0   
9997   0    0     0        0    0    0    0    0    0    0  ...    0     0   
9998   0    0     0        0    0    0    0    0    0    0  ...    0     0   
9999   0    0     0        0    0    0    0    0    0    0  ...    0     0   

      țo  țr  țz  țó  țăêvH  punc_density  vowels_density  n

In [12]:
# Normalizare a datelor
scale_train, scale_validation = train_df, valid_df

#scale_train, scale_validation = normalize_data(train_df, valid_df, "l1")

In [13]:
#print(scale_train)

In [14]:
svm_classifier = svm.SVC(C=3, kernel='linear')

svm_classifier.fit(scale_train, y_train)
predicted = svm_classifier.predict(scale_validation)

print(accuracy_score(predicted, y_valid))

0.5906


In [15]:
def procces_merged(x_merged):
    count_vect = CountVectorizer(lowercase = False, max_features=3000, strip_accents=None, min_df=1, stop_words=None)
    x_merged_counts = count_vect.fit_transform(x_merged)
    data = x_merged_counts.toarray()
    
    names = count_vect.get_feature_names_out()
    
    merged_df = final_features(x_merged, data, names)
    
    scale_merged, scale_merged = normalize_data(merged_df, merged_df, "l2")
    
    return scaled_merged

In [16]:
# print(predicted)
# print(predicted.shape)
# print(x_valid.shape)

In [17]:
import csv

def write_prediction(test_data, predicted):
    header = ['id', 'label']
    
    data = []
    
    for i in range(0, len(test_data)):
        id = int(test_data[i][0])
        label = int(predicted[i])
        
        data.append([id, label])
        
    with open('submisie', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        
        for item in data:
            writer.writerow(item)

In [18]:
# Citire date test

test_data_init = np.genfromtxt('data_aliens/test_samples.txt', dtype='str', comments='\\\\\\\\\\\\\\\\', delimiter='\t', invalid_raise = False)

x_test = get_sentence(test_data_init)

X_test_counts = count_vect.transform(x_test)

test_data = X_test_counts.toarray()

test_df = final_features(x_test, test_data, names)

DF       00  000  0002  000g001  001  002  003  004  005  006  ...  țkK  țlrȚ  \
0      0    0     0        0    0    0    0    0    0    0  ...    0     0   
1      0    0     0        0    0    0    0    0    0    0  ...    0     0   
2      0    0     0        0    0    0    0    0    0    0  ...    0     0   
3      0    0     0        0    0    0    0    0    0    0  ...    0     0   
4      0    0     0        0    0    0    0    0    1    0  ...    0     0   
...   ..  ...   ...      ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
4995   0    0     0        0    0    0    0    0    0    0  ...    0     0   
4996   0    0     0        0    0    0    0    0    0    0  ...    0     0   
4997   0    0     0        0    0    0    0    0    0    0  ...    0     0   
4998   0    0     0        0    0    0    0    0    0    0  ...    0     0   
4999   0    0     0        0    0    0    0    0    0    0  ...    0     0   

      țo  țr  țz  țó  țăêvH  punc_density  vowels_density  n

In [19]:
#scale_test, scale_test = normalize_data(test_df, test_df, "l1")
scale_test = test_df

In [20]:
count_vect = CountVectorizer(lowercase = False, max_features=3000, strip_accents=None, min_df=1, stop_words=None)
x_merged_counts = count_vect.fit_transform(x_merged)
data = x_merged_counts.toarray()

names = count_vect.get_feature_names_out()

In [21]:
merged_df = final_features(x_merged, data, names)
    
# scaled_merged, scaled_merged = normalize_data(merged_df, merged_df, "l2")

DF        00  000  0001  0002  000g001  001  002  003  004  005  ...  țkK  țlrȚ  \
0       0    0     0     0        0    0    0    0    0    0  ...    0     0   
1       0    0     0     0        0    0    0    0    0    0  ...    0     0   
2       1    0     0     0        0    0    0    0    0    0  ...    0     0   
3       0    0     0     0        0    0    0    0    0    0  ...    0     0   
4       0    0     0     0        0    0    0    0    0    0  ...    0     0   
...    ..  ...   ...   ...      ...  ...  ...  ...  ...  ...  ...  ...   ...   
14995   0    0     0     0        0    0    0    0    0    0  ...    0     0   
14996   0    0     0     0        0    0    0    0    0    0  ...    0     0   
14997   0    0     0     0        0    0    0    0    0    0  ...    0     0   
14998   0    0     0     0        0    0    0    0    0    0  ...    0     0   
14999   0    0     0     0        0    0    0    0    0    0  ...    0     0   

       țo  țr  țz  țó  țăêvH  punc_d

In [22]:
#scaled_merged, scaled_merged = normalize_data(merged_df, merged_df, "l1")
scaled_merged = merged_df

In [23]:
# svm_classifier = svm.SVC(C=3, kernel='linear')
# svm_classifier.fit(scaled_merged, y_merged)

predicted = svm_classifier.predict(scale_test)
print("Done!")

Done!


In [24]:
write_prediction(test_data_init, predicted)

In [25]:
print(test_data_init.shape)
print(predicted.shape)

(5000, 2)
(5000,)
